# 날씨 AI 챗봇 시스템 구축 강의

##  강의 개요
이 노트북에서는 Python을 사용하여 날씨 정보를 실시간으로 조회하고 대화할 수 있는 AI 채팅 시스템을 구축하는 방법을 단계별로 학습합니다.

##  학습 목표
- LangChain MCP(Model Control Protocol) 클라이언트 활용법
- Ollama를 통한 로컬 LLM 모델 활용
- 비동기 프로그래밍과 도구 호출(Tool Calling) 구현
- 대화형 AI 시스템의 전체 아키텍처 이해

---

##  **1. 필수 라이브러리 임포트**

시스템 구축에 필요한 핵심 라이브러리들을 임포트합니다.

- `asyncio`: 비동기 프로그래밍 지원
- `MultiServerMCPClient`: MCP 서버와의 통신 담당
- `ollama`: 로컬 LLM 모델 사용
- `json`: 데이터 파싱 및 처리



In [1]:
import asyncio
from langchain_mcp_adapters.client import MultiServerMCPClient
import json
import ollama

##  **2. 전역 설정 및 변수 초기화**

시스템에서 사용할 모델명과 대화 기록을 저장할 변수를 설정합니다.

- `MODEL_NAME`: 사용할 Ollama 모델 지정 (llama3.1 모델 사용)
- `conversation_history`: 대화 내역을 저장하는 빈 리스트로 초기화, AI와의 모든 대화가 여기에 누적됨



In [2]:
MODEL_NAME = "llama3.1"
conversation_history = []

## **3. MCP 도구 호출 함수 구현**

MCP 클라이언트를 통해 특정 도구를 호출하는 핵심 함수입니다.

- 사용 가능한 도구 목록 조회
- 특정 도구 존재 여부 확인
- 비동기 도구 실행
- 에러 핸들링

In [3]:
async def call_mcp_tool(client, tool_name, tool_args):
    tools = client.get_tools()
    tool = next((t for t in tools if t.name == tool_name), None)
    if tool is None:
        raise ValueError(f"Tool '{tool_name}' not found")
    return await tool.coroutine(**tool_args)

##  **4. 날씨 정보 조회 함수**

MCP를 통해 실제 날씨 데이터를 조회하는 함수를 구현합니다.

- `get_todays_weather` 도구 활용
- 다양한 반환 타입 처리 (tuple, 단일 값)
- 한국어 에러 메시지 제공


In [4]:
async def get_weather_mcp(client, city_name):
    try:
        result = await call_mcp_tool(client, "get_todays_weather", {"city_name": city_name})
        if isinstance(result, tuple):
            weather_data = result[0]
        else:
            weather_data = result
        return weather_data
    except Exception as e:
        return f"날씨 조회 오류: {str(e)}"

##  **5. 도구 스키마 정의**

Ollama가 날씨 도구를 인식하고 사용할 수 있도록 스키마를 정의합니다.

- 함수 타입과 이름 정의
- 함수 기능에 대한 명확한 설명
- 매개변수 타입과 설명
- 필수 매개변수 지정

In [5]:
def get_tools():
    return [{
        "type": "function",
        "function": {
            "name": "get_todays_weather",
            "description": "특정 도시의 현재 날씨 정보를 조회합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city_name": {
                        "type": "string",
                        "description": "날씨를 조회할 도시 이름",
                    }
                },
                "required": ["city_name"],
            },
        },
    }]

##  **6. 날씨 쿼리 감지 시스템**
사용자의 입력이 날씨 관련 질문인지 자동으로 판단하는 시스템입니다.

**감지 키워드:**
- **한국어**: 날씨, 기온, 온도, 비, 눈, 바람, 습도 등
- **영어**: weather, temperature, rain, snow, wind 등
- **상태 키워드**: 따뜻, 춥, 덥, 시원 등

**동작 원리:**
- 입력 텍스트를 소문자로 변환
- 키워드 리스트와 매칭 검사
- 하나라도 포함되면 날씨 쿼리로 판단

In [6]:
def is_weather_query(text):
    weather_keywords = [
        "날씨", "기온", "온도", "비", "눈", "바람", "습도", "맑음", "흐림",
        "weather", "temperature", "rain", "snow", "wind", "sunny", "cloudy",
        "따뜻", "춥", "덥", "시원"
    ]
    text_lower = text.lower()
    return any(keyword in text_lower for keyword in weather_keywords)

##  **7. 비동기 도구 호출 처리**

Ollama의 도구 호출 결과를 비동기적으로 처리하는 복합 함수입니다.

**처리 단계:**
1. 도구 호출 존재 여부 확인
2. 각 도구 호출 정보 파싱 (함수명, 인자)
3. JSON 형태의 인자 처리
4. 실제 날씨 데이터 조회 실행
5. 결과를 표준 포맷으로 변환

---

In [7]:
async def process_tool_calls_async(message, mcp_client):
    if not hasattr(message, "tool_calls") or not message.tool_calls:
        return []
    
    tool_results = []
    for i, tool_call in enumerate(message.tool_calls):
        try:
            function = tool_call.function
            function_name = getattr(function, "name", None)
            arguments = getattr(function, "arguments", None)
            
            if function_name != "get_todays_weather":
                continue
            
            city_name = None
            if isinstance(arguments, dict):
                city_name = arguments.get("city_name")
            elif isinstance(arguments, str):
                try:
                    args_dict = json.loads(arguments)
                    city_name = args_dict.get("city_name")
                except:
                    city_name = arguments
            
            if not city_name:
                continue
            
            print(f" {city_name} 날씨 조회 중...")
            result = await get_weather_mcp(mcp_client, city_name)
            tool_call_id = getattr(tool_call, "id", f"call_{i}")
            
            tool_results.append({
                "role": "tool",
                "tool_call_id": tool_call_id,
                "name": function_name,
                "content": str(result),
            })
            
        except Exception as e:
            print(f" 오류: {e}")
    
    return tool_results

##  **8. 메인 AI 채팅 함수**

사용자와 AI 간의 대화를 관리하는 핵심 함수입니다.

1. 사용자 메시지를 대화 기록에 추가
2. 날씨 관련 질문인지 판단
3. 날씨 쿼리면 도구 호출 모드로 실행
4. 도구 호출 결과 처리 후 자연어 응답 생성


In [8]:
async def chat_with_ai(user_message, mcp_client):
    global conversation_history
    
    conversation_history.append({"role": "user", "content": user_message})
    
    try:
        if is_weather_query(user_message):
            print(" 날씨 도구 사용 중...")
            
            response = ollama.chat(
                model=MODEL_NAME,
                messages=conversation_history,
                options={"temperature": 0.1},
                tools=get_tools(),
            )
            
            message = response["message"]
            
            if hasattr(message, "tool_calls") and message.tool_calls:
                tool_results = await process_tool_calls_async(message, mcp_client)
                
                if tool_results:
                    assistant_message = {"role": "assistant", "content": message.content or ""}
                    if hasattr(message, "tool_calls"):
                        tool_calls_data = []
                        for tc in message.tool_calls:
                            if hasattr(tc, "function"):
                                tool_calls_data.append({
                                    "id": getattr(tc, "id", "unknown"),
                                    "type": "function",
                                    "function": {
                                        "name": tc.function.name,
                                        "arguments": tc.function.arguments,
                                    },
                                })
                        assistant_message["tool_calls"] = tool_calls_data
                    
                    conversation_history.append(assistant_message)
                    
                    for result in tool_results:
                        conversation_history.append(result)
                    
                    final_response = ollama.chat(
                        model=MODEL_NAME,
                        messages=conversation_history,
                        options={"temperature": 0.7},
                    )
                    
                    final_content = final_response["message"]["content"]
                    conversation_history.append({"role": "assistant", "content": final_content})
                    return final_content
                else:
                    content = message.content or "죄송합니다. 날씨 정보를 가져올 수 없습니다."
                    conversation_history.append({"role": "assistant", "content": content})
                    return content
            else:
                content = message.content or "구체적인 도시명을 알려주세요."
                conversation_history.append({"role": "assistant", "content": content})
                return content
        
        response = ollama.chat(
            model=MODEL_NAME,
            messages=conversation_history,
            options={"temperature": 0.7},
        )
        
        content = response["message"]["content"]
        conversation_history.append({"role": "assistant", "content": content})
        return content
        
    except Exception as e:
        error_msg = f"오류: {str(e)}"
        conversation_history.append({"role": "assistant", "content": error_msg})
        return error_msg

##  **9. 기본 채팅 인터페이스 구현**

사용자와 실시간으로 상호작용할 수 있는 채팅 인터페이스를 구현합니다.

**주요 기능:**
- MCP 연결: `http://localhost:8005/sse`로 날씨 서버 연결
- 명령어 처리: quit/clear 등
- 실시간 대화 및 에러 복구

---

In [9]:
async def start_chat():
    print(" 날씨 AI 어시스턴트")
    print("종료: 'quit' | 초기화: 'clear'")
    print("=" * 40)
    
    async with MultiServerMCPClient(
        {
            "weather": {
                "url": "http://localhost:8005/sse",
                "transport": "sse",
            }
        }
    ) as client:
        
        while True:
            try:
                user_input = input(" 당신: ").strip()
                
                if user_input.lower() in ['quit', 'exit', '종료', 'q']:
                    print("프로그램 종료")
                    break
                
                if user_input.lower() in ['clear', '초기화']:
                    global conversation_history
                    conversation_history = []
                    print(" 대화 기록 초기화됨")
                    continue
                
                if not user_input:
                    continue
                
                response = await chat_with_ai(user_input, client)
                print(f" AI: {response}")
                print("-" * 40)
                
            except KeyboardInterrupt:
                print("\n 프로그램 종료")
                break
            except Exception as e:
                print(f" 오류: {e}")

## 

In [11]:
# 바로 채팅 시작
try:
    import nest_asyncio
    nest_asyncio.apply()
    loop = asyncio.get_event_loop()
    loop.run_until_complete(start_chat())
except:
    asyncio.run(start_chat())

 날씨 AI 어시스턴트
종료: 'quit' | 초기화: 'clear'


 당신:  오늘 서울 날씨를 알려줘


 날씨 도구 사용 중...
 서울 날씨 조회 중...
 AI: 서울의 현재 기상은 다음과 같습니다.

*   기온: 10도
*   습도: 60%
*   비/눈: 없음
*   바람: 약한 바람이吹고 있습니다.

기상 조건은 항시 업데이트 되므로 정확도가 유지됩니다.
----------------------------------------


 당신:  Tell me the weather in Seoul today


 날씨 도구 사용 중...
 Seoul 날씨 조회 중...
 AI: 서울의 현재 날씨는 주로 구름이 내리는 형태입니다.
기온은 32도이며, 습도는 71%에 불과합니다.
풍향은 서남서쪽이고, 풍속은 약한 바람으로 측정됩니다.
대기 압력은 1011hPa이며, 대기 가시성은 10km입니다.
----------------------------------------


 당신:  quit


프로그램 종료


##  **10. 언어 감지 시스템**

입력 텍스트가 한국어인지 영어인지 자동으로 감지하는 시스템입니다.

**감지 알고리즘:**
1. 한국어 문자 범위 검사 (가-힣)
2. 전체 문자 중 한국어 비율 계산
3. 30% 이상이면 한국어로 판단

In [12]:
import re
from langchain_community.llms import Ollama as OllamaLLM

# 한국어 번역용 모델 (한국어를 잘 이해하는 모델)
korean_llm = OllamaLLM(model="exaone3.5:7.8b", temperature=0)

def detect_language(text):
    """텍스트가 한국어인지 영어인지 감지"""
    korean_chars = re.findall(r'[가-힣]', text)
    total_chars = len(re.findall(r'[a-zA-Z가-힣]', text))
    
    if total_chars == 0:
        return "unknown"
    
    korean_ratio = len(korean_chars) / total_chars
    return "korean" if korean_ratio > 0.3 else "english"

# 테스트해보기
test_texts = [
    "서울 날씨 어때?",
    "What's the weather in Seoul?",
    "오늘 비 와?",
    "Is it raining today?"
]

for text in test_texts:
    lang = detect_language(text)
    print(f"'{text}' -> {lang}")

'서울 날씨 어때?' -> korean
'What's the weather in Seoul?' -> english
'오늘 비 와?' -> korean
'Is it raining today?' -> english


## **11. 한국어→영어 번역 시스템**

한국어 입력을 영어로 번역하여 영어 기반 AI 모델과 소통할 수 있게 합니다.

In [14]:
def translate_to_english(korean_text):
    """한국어를 영어로 번역"""
    try:
        prompt = f"""다음 한국어 문장을 자연스러운 영어로 번역해주세요. 번역된 영어만 출력하고 다른 설명은 하지 마세요.

한국어: {korean_text}
영어:"""
        
        print(f" 번역 중: '{korean_text}'")
        english_translation = korean_llm.invoke(prompt).strip()
        print(f" 번역 결과: '{english_translation}'")
        return english_translation
        
    except Exception as e:
        print(f" 번역 오류: {e}")
        return korean_text  # 번역 실패 시 원본 반환

# 테스트해보기
korean_questions = [
    "서울 날씨 어때?",
    "내일 비 올까?",
    "부산 기온이 얼마야?",
    "오늘 우산 가져가야 해?"
]

for q in korean_questions:
    translate_to_english(q)
    print("-" * 40)

 번역 중: '서울 날씨 어때?'
 번역 결과: 'What's the weather like in Seoul?'
----------------------------------------
 번역 중: '내일 비 올까?'
 번역 결과: 'Will it rain tomorrow?'
----------------------------------------
 번역 중: '부산 기온이 얼마야?'
 번역 결과: 'What's the temperature in Busan?'
----------------------------------------
 번역 중: '오늘 우산 가져가야 해?'
 번역 결과: 'Should I bring an umbrella today?'
----------------------------------------


##  **12. 영어→한국어 번역 시스템**

AI 응답을 한국어 사용자가 이해할 수 있도록 번역하는 시스템입니다.

In [15]:
def translate_to_korean(english_text):
    """영어를 한국어로 번역 (AI 응답용)"""
    try:
        prompt = f"""다음 영어 문장을 자연스러운 한국어로 번역해주세요. 번역된 한국어만 출력하고 다른 설명은 하지 마세요.

영어: {english_text}
한국어:"""
        
        korean_translation = korean_llm.invoke(prompt).strip()
        print(f" 영어→한국어 번역: '{english_text}' → '{korean_translation}'")
        return korean_translation
        
    except Exception as e:
        print(f" 번역 오류: {e}")
        return english_text  # 번역 실패 시 원본 반환

# 테스트해보기
english_responses = [
    "The weather in Seoul is sunny with a temperature of 25°C.",
    "It will rain tomorrow afternoon.",
    "The current temperature in Busan is 28°C."
]

for response in english_responses:
    translate_to_korean(response)
    print("-" * 40)

 영어→한국어 번역: 'The weather in Seoul is sunny with a temperature of 25°C.' → '서울 날씨는 맑고 기온이 25°C입니다.'
----------------------------------------
 영어→한국어 번역: 'It will rain tomorrow afternoon.' → '내일 오후에 비가 올 거예요.'
----------------------------------------
 영어→한국어 번역: 'The current temperature in Busan is 28°C.' → '부산의 현재 기온은 28°C입니다.'
----------------------------------------


##  **13. 번역 지원 채팅 인터페이스**

다국어 번역 기능이 통합된 완성형 채팅 시스템입니다.
- 🇰🇷/🇺🇸 언어 감지 및 표시
- 실시간 번역 진행 상황 표시


In [1]:
async def chat_with_ai(user_message, mcp_client):
    global conversation_history
    
    original_message = user_message
    detected_lang = detect_language(user_message)
    
    if detected_lang == "korean":
        print("🇰🇷 한국어 감지! 영어로 번역합니다...")
        user_message = translate_to_english(user_message)
    else:
        print("🇺🇸 영어 입력으로 진행합니다...")
    
    conversation_history.append({"role": "user", "content": user_message})
    
    try:
        if is_weather_query(original_message):  # 원본 메시지로 날씨 쿼리 판단
            print("🔧 날씨 도구 사용 중...")
            
            response = ollama.chat(
                model=MODEL_NAME,
                messages=conversation_history,
                options={"temperature": 0.1},
                tools=get_tools(),
            )
            
            message = response["message"]
            
            if hasattr(message, "tool_calls") and message.tool_calls:
                tool_results = await process_tool_calls_async(message, mcp_client)
                
                if tool_results:
                    assistant_message = {"role": "assistant", "content": message.content or ""}
                    if hasattr(message, "tool_calls"):
                        tool_calls_data = []
                        for tc in message.tool_calls:
                            if hasattr(tc, "function"):
                                tool_calls_data.append({
                                    "id": getattr(tc, "id", "unknown"),
                                    "type": "function",
                                    "function": {
                                        "name": tc.function.name,
                                        "arguments": tc.function.arguments,
                                    },
                                })
                        assistant_message["tool_calls"] = tool_calls_data
                    
                    conversation_history.append(assistant_message)
                    
                    for result in tool_results:
                        conversation_history.append(result)
                    
                    final_response = ollama.chat(
                        model=MODEL_NAME,
                        messages=conversation_history,
                        options={"temperature": 0.7},
                    )
                    
                    final_content = final_response["message"]["content"]
                    conversation_history.append({"role": "assistant", "content": final_content})
                    
                    #  새로운 기능: 한국어로 질문했으면 답변도 한국어로 번역
                    if detected_lang == "korean":
                        print("🇰🇷 답변을 한국어로 번역합니다...")
                        final_content = translate_to_korean(final_content)
                    
                    return final_content
                else:
                    content = message.content or "Sorry, I couldn't get weather information."
                    if detected_lang == "korean":
                        content = translate_to_korean(content)
                    conversation_history.append({"role": "assistant", "content": content})
                    return content
            else:
                content = message.content or "Please provide a specific city name."
                if detected_lang == "korean":
                    content = translate_to_korean(content)
                conversation_history.append({"role": "assistant", "content": content})
                return content
        
        # 일반 대화
        response = ollama.chat(
            model=MODEL_NAME,
            messages=conversation_history,
            options={"temperature": 0.7},
        )
        
        content = response["message"]["content"]
        conversation_history.append({"role": "assistant", "content": content})
        
        # 새로운 기능: 한국어로 질문했으면 답변도 한국어로 번역
        if detected_lang == "korean":
            print("🇰🇷 답변을 한국어로 번역합니다...")
            content = translate_to_korean(content)
        
        return content
        
    except Exception as e:
        error_msg = f"오류: {str(e)}"
        conversation_history.append({"role": "assistant", "content": error_msg})
        return error_msg

In [ ]:
async def start_chat():
    print("번역 기능이 추가된 날씨 AI 어시스턴트")
    print("종료: 'quit' | 초기화: 'clear'")
    print("=" * 50)
    
    async with MultiServerMCPClient(
        {
            "weather": {
                "url": "http://localhost:8005/sse",
                "transport": "sse",
            }
        }
    ) as client:
        
        while True:
            try:
                user_input = input(" 당신: ").strip()
                
                if user_input.lower() in ['quit', 'exit', '종료', 'q']:
                    print(" 프로그램 종료")
                    break
                
                if user_input.lower() in ['clear', '초기화']:
                    global conversation_history
                    conversation_history = []
                    print(" 대화 기록 초기화됨")
                    continue
                
                if not user_input:
                    continue
                
                response = await chat_with_ai(user_input, client)
                print(f"🤖 AI: {response}")
                print("-" * 50)
                
            except KeyboardInterrupt:
                print("\n 프로그램 종료")
                break
            except Exception as e:
                print(f" 오류: {e}")


await start_chat()

##  **실습: 키워드 확장**

현재 `is_weather_query()` 함수의 날씨 키워드 리스트에 다음을 추가하세요:

**테스트:** "오늘 우산 가져가야 하나요?" 입력 시 날씨 도구가 활성화되는지 확인하세요.

In [ ]:
def is_weather_query(text):
    weather_keywords = [
        "날씨", "기온", "온도", "비", "눈", "바람", "습도", "맑음", "흐림",
        "weather", "temperature", "rain", "snow", "wind", "sunny", "cloudy",
        "따뜻", "춥", "덥", "시원"
        # 여기에 추가하세요:
        
        
    ]
    text_lower = text.lower()
    return any(keyword in text_lower for keyword in weather_keywords)

In [19]:
#테스트
await start_chat()

🌤️ 번역 기능이 추가된 날씨 AI 어시스턴트
✅ 한국어/영어 모두 지원!
종료: 'quit' | 초기화: 'clear'


👤 당신:  quit


👋 프로그램 종료
